In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [9]:
#functions to extract data

def get_title(soup):
    try:
        title = soup.find('span', attrs={'id':"productTitle"}).text.strip()
        
    except AttributeError:
        title = ""
        
    return title;

def get_price(soup):
    try:
        price_elem = soup.find("span", attrs={"class":"a-price a-text-normal aok-align-center reinventPriceAccordionT2"})
        if price_elem is not None:
            price = price_elem.find("span", attrs={"class":"a-offscreen"}).text.strip()
        else:
            price = ""

    except AttributeError:
        try:
            price_elem = soup.find("span", attrs={"class":"a-price aok-align-center reinventPricePriceToPayMargin priceToPay"})
            if price_elem is not None:
                price = price_elem.find("span", attrs={"class":"a-offscreen"}).text.strip()
            else:
                price = ""
            
        except:
            price = ""

    return price

    
def get_rating(soup):
    try:
        rating = soup.find("span", attrs={"class":"a-size-base a-color-base"}).text.strip()
        
    except AttributeError:
        try:
            rating = soup.find("span", attrs={"class":"a-icon-alt"}).text.strip()
        
        except:
            rating = ""
    
    return rating

def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span", attrs={"class":"a-size-base a-color-price a-text-bold"}).string.strip()

    except AttributeError:
        available = "Not Available"	

    return available


In [10]:
if __name__ == '__main__':
    
    #USER AGENT GOES HERE
    HEADERS = ({'User-Agent':"***********************", 'Accept-Language': 'en-US, en;q=0.5'})
    
    #webpage URL
    URL = "https://www.amazon.com/s?k=fifa+2023&crid=2J7RXCD8IDH7T&sprefix=fifa+2023%2Caps%2C131&ref=nb_sb_noss_1"
    
    #http request
    webpage = requests.get(URL, headers=HEADERS)
    
    #Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    
    #fetch links in a list
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
    
    #creating a list to store all the links
    list_of_links = []
    
    for link in links:
        list_of_links.append(link.get('href'))
        
    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

    for link in list_of_links:
        webpage_1 = requests.get("https://www.amazon.com" + link, headers=HEADERS)
        
        soup_1 = BeautifulSoup(webpage_1.content, "html.parser")
        
        # calling functions to populate dictionary
        d['title'].append(get_title(soup_1))
        d['price'].append(get_price(soup_1))
        d['rating'].append(get_rating(soup_1))
        d['reviews'].append(get_review_count(soup_1))
        d['availability'].append(get_availability(soup_1))
        
    amazon_fifa_df = pd.DataFrame(d)
    amazon_fifa_df['title'].replace('', np.nan, inplace=True)
    amazon_fifa_df = amazon_fifa_df.dropna(subset=['title'])
    amazon_fifa_df.to_csv("amazon_fifa_data.csv", header=True, index=False)

In [12]:
amazon_fifa_df

,title,price,rating,reviews,availability
0,FIFA 23 - For PlayStation 4,$37.60,,579 ratings,Only 9 left in stock - order soon
1,FIFA 23 Standard Edition Playstation 5 (PS5)| ...,,,"1,657 ratings",Not Available
3,FIFA 23 Standard Edition XBOX ONE | English | ...,,,"1,120 ratings",Not Available
4,FIFA 23 (Legacy Edition) - For Nintendo Switch,,,135 ratings,Only 6 left in stock - order soon
5,FIFA 23 Legacy Edition (Switch) Import Region ...,$32.85,,"1,605 ratings",Not Available
6,FIFA 23 - PlayStation 5,,,"1,122 ratings",Only 1 left in stock - order soon
7,FIFA 23 - Xbox Series X,$29.88,,178 ratings,Only 4 left in stock - order soon
9,FIFA 23 Standard Edition PS4 | English | Impor...,,,"2,586 ratings",Not Available
10,FIFA 21 – PlayStation 4 & PlayStation 5,,"Get Fast, Free Shipping with Amazon Prime","6,591 ratings",Not Available
12,EA SPORTS FIFA 23 Nintendo Switch Legacy Editi...,,,25 ratings,Not Available
